# **Phase 3**

In [199]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [200]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras import optimizers

# Set random seed for reproducability
tensorflow.random.set_seed(0)

print("Libraries imported.")
print('Keras version:',keras.__version__)
print('TensorFlow version:',tensorflow.__version__)

Libraries imported.
Keras version: 3.7.0
TensorFlow version: 2.18.0


In [201]:
TRAIN_PATH = 'train_data.csv'
TEST_PATH = 'test_data.csv'

In [202]:
train = pd.read_csv('train_data.csv')

In [203]:
test = pd.read_csv('test_data.csv')

In [204]:
X_train = train.drop(columns=['isFraud'])
y_train = train['isFraud']
X_train.shape, y_train.shape

((10167052, 15), (10167052,))

In [205]:
y_train.value_counts()

isFraud
0    5083526
1    5083526
Name: count, dtype: int64

In [206]:

X_test = test.drop(columns=['isFraud'])
y_test = test['isFraud']
X_test.shape, y_test.shape

((1272524, 15), (1272524,))

In [207]:
y_test.value_counts()

isFraud
0    1270881
1       1643
Name: count, dtype: int64

In [208]:
X_train, X_unused, y_train, y_unused = train_test_split(X_train, y_train, test_size=0.95, stratify=y_train, random_state=777)

In [209]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.90, stratify=y_test, random_state=777)

In [210]:
# Set data types for float features
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

# Set data types for categorical labels
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
y_val = utils.to_categorical(y_val)

y_train = np.argmax(y_train, axis=1)  # Use this if y_train is one-hot encoded
y_test = np.argmax(y_test, axis=1)
y_val = np.argmax(y_val, axis=1)

### Define Neural Network

In [211]:
num_features = X_train.shape[1]
num_classes = 2

In [212]:
# Define a classifier network
hl_neurons = 20 # Number of hidden layer nodes

mlp = Sequential()
mlp.add(Dense(hl_neurons, input_dim=num_features, activation='relu'))
mlp.add(Dense(hl_neurons, input_dim=hl_neurons, activation='relu'))
mlp.add(Dense(1, input_dim=hl_neurons, activation='sigmoid'))

print(mlp.summary())

c:\Users\n01715077\Downloads\prepstone-final\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 20)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 761 (2.97 KB)

 Trainable params: 761 (2.97 KB)

 Non-trainable params: 0 (0.00 B)

None


In [214]:
#hyper-parameters
lr = 0.001
num_epochs = 5

opt = optimizers.Adam(learning_rate=lr)

mlp.compile(loss='binary_crossentropy',
            optimizer=opt,
            metrics=['precision']
            )

# Train the model over 50 epochs using 10-observation batches and using the test holdout dataset for validation

history = mlp.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/5
15886/15886 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - loss: 1863.1514 - precision: 0.6058 - val_loss: 3197.6606 - val_precision: 0.0015
Epoch 2/5
15886/15886 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 577.0080 - precision: 0.6511 - val_loss: 190.5564 - val_precision: 0.0043
Epoch 3/5
15886/15886 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 353.1159 - precision: 0.6659 - val_loss: 66.6694 - val_precision: 0.0055
Epoch 4/5
15886/15886 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - loss: 154.7311 - precision: 0.6845 - val_loss: 55.3111 - val_precision: 0.0055
Epoch 5/5
15886/15886 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 51.7081 - precision: 0.6870 - val_loss: 0.6665 - val_precision: 0.0015


In [ ]:
y_pred = mlp.predict(X_test)

35790/35790 ━━━━━━━━━━━━━━━━━━━━ 20s 551us/step


In [198]:
print(y_pred)
print(y_test)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [1.]
 [1.]]
[0 0 0 ... 0 0 0]


In [122]:
print(y_pred)

[[0.50268686 0.49731317]
 [0.50268686 0.49731317]
 [0.50268686 0.49731317]
 ...
 [0.50268686 0.49731317]
 [0.50268686 0.49731317]
 [0.50268686 0.49731317]]


In [129]:
# Convert one-hot encoded labels to class indices
y_test_indices = tf.argmax(y_test, axis=1).numpy()
y_pred_indices = tf.argmax(y_pred, axis=1).numpy()

In [125]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [130]:
cm = confusion_matrix(y_test_indices, y_pred_indices)

In [132]:
print(cm)

[[1143793       0]
 [   1479       0]]
